In [1]:
import sys
sys.path.append("../")
from dotenv import load_dotenv
load_dotenv("/Users/vigil/Desktop/Vigil/autoredteam/autoredteam/.env")

True

In [2]:
import matplotlib.pyplot as plt
from collections import Counter

def get_detectors(test_detector_dict):
    extracted_values = {}
    for key, values in test_detector_dict.items():
        extracted_values[key] = [f"{value.__class__.__module__}.{value.__class__.__name__}" for value in values]
    return extracted_values


def get_counts(extracted_values):
    values = [value for sublist in extracted_values.values() for value in sublist]
    value_counts = Counter(values)

    return value_counts

In [2]:
from autoredteam.agents.octo import OctoAPI
agent = OctoAPI(name = "mistral-7b-instruct-fp16", generations=2)

Loading OctoAI Agent: mistral-7b-instruct-fp16


In [3]:
from autoredteam.harnesses.dimension import SecurityHarness
# , ToxicityHarness, PrivacyHarness, HallucinationHarness, RobustnessHarness, ToxicityHarness, StereotypeHarness, FairnessHarness, EthicsHarness
harnesses = [SecurityHarness]
harness_instances = [harness(agent) for harness in harnesses]

: 

In [ ]:
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder

# Initialize a dictionary to store the nodes and their parents
nodes = defaultdict(list)

# Iterate over the harness instances
for i in harness_instances:
    # Get the class name and module of the harness instance
    i_name = i.__class__.__name__
    i_module = i.__class__.__module__.split('.')[-1]

    # Iterate over the test instances in each harness instance
    for j in i.test_instances:
        # Get the class name and module of the test instance
        j_name = j.__class__.__name__
        j_module = j.__class__.__module__.split('.')[-1]

        # Add the test instance as a child of the harness instance
        nodes[f'{i_module}.{i_name}'].append(f'{j_module}.{j_name}')

        # Iterate over the detectors in each test instance
        for k in j.detectors:
            # Get the class name of the detector
            k_name = k.__class__.__name__

            # Add the detector as a child of the test instance
            nodes[f'{j_module}.{j_name}'].append(k_name)

# Create a list of tuples representing the nodes and their parents
data = [(parent, child) for parent in nodes for child in nodes[parent]]

# Flatten the data and encode the strings to numerical identifiers
le = LabelEncoder()
data_flat = le.fit_transform([item for sublist in data for item in sublist])

# Reshape the data to its original shape
data_encoded = data_flat.reshape(-1, 2)

# Create a linkage matrix
linkage_matrix = linkage(data_encoded, 'single')

# Create labels for the pairs of data points
labels = [f'{parent} -> {child}' for parent, child in data]

# Create the dendrogram
plt.figure(figsize=(10, 7))
dendrogram(linkage_matrix, labels=labels, orientation='right')
plt.show()

In [ ]:
from ete3 import Tree, TreeStyle, TextFace
from collections import defaultdict
# Initialize a dictionary to store the nodes and their parents
nodes = defaultdict(list)

# Iterate over the harness instances
for i in harness_instances:
    # Get the class name and module of the harness instance
    i_name = i.__class__.__name__
    i_module = i.__class__.__module__.split('.')[-1]

    # Iterate over the test instances in each harness instance
    for j in i.test_instances:
        # Get the class name and module of the test instance
        j_name = j.__class__.__name__
        j_module = j.__class__.__module__.split('.')[-1]

        # Add the test instance as a child of the harness instance
        nodes[f'{i_module}.{i_name}'].append(f'{j_module}.{j_name}')

        # Iterate over the detectors in each test instance
        for k in j.detectors:
            # Get the class name of the detector
            k_name = k.__class__.__name__

            # Add the detector as a child of the test instance
            nodes[f'{j_module}.{j_name}'].append(k_name)

# Create a new tree
t = Tree()

# Add nodes to the tree
for parent, children in nodes.items():
    parent_node = t.search_nodes(name=parent)
    if not parent_node:
        parent_node = t.add_child(name=parent)
    else:
        parent_node = parent_node[0]
    for child in children:
        parent_node.add_child(name=child)

# Create a tree style
ts = TreeStyle()
ts.show_leaf_name = False

# Add labels to the nodes
for node in t.traverse():
    node.add_face(TextFace(node.name), column=0, position="branch-right")

# Show the tree
t.render("%%inline", tree_style=ts)

In [ ]:
!pip3 install PyQt5

DEPRECATION: Loading egg at /Users/vigil/anaconda3/envs/art/lib/python3.11/site-packages/ete3-3.1.3-py3.10.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [ ]:
tests = [instance.test_instances for instance in harness_instances]

In [ ]:
tests

In [ ]:
# Create a dictionary for each element in sec_tests and store the dictionaries in another list
test_detector_dicts = [{str(test): test.detectors for test in sec_test} for sec_test in tests]

In [ ]:
test_detector_dicts[0]

In [ ]:
keys_list = list(test_detector_dicts[1].keys())
tester = keys_list[0]


In [ ]:
import os
import importlib.util
import inspect
import sys

sys.path.append('/Users/vigil/Desktop/Vigil/autoredteam')

# Define the directory with the .py files
directory = '/Users/vigil/Desktop/Vigil/autoredteam/autoredteam/tests'

# Define the markdown file
markdown_directory = 'markdown_files'
markdown_file = os.path.join(markdown_directory, 'attributes.md')

# Create the directory if it does not exist
os.makedirs(markdown_directory, exist_ok=True)
# Open the markdown file
with open(markdown_file, 'w') as f:
    # List all .py files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.py'):
            # Import the .py file as a module
            spec = importlib.util.spec_from_file_location(filename[:-3], os.path.join(directory, filename))
            module = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(module)

            # Get the attributes of the module
            attributes = inspect.getmembers(module)

            # Filter the attributes
            attributes = [attr for attr in attributes if attr[0] in ['__doc__', 'detectors', 'uri', 'name', 'description', 'tags', 'goal']]

            # Write the attributes to the markdown file in a table format
            f.write(f'## {filename[:-3]}\n')
            f.write('| Attribute | Value |\n')
            f.write('| --- | --- |\n')
            for attr in attributes:
                f.write(f'| {attr[0]} | {attr[1]} |\n')
            f.write('\n')

In [ ]:
import os
from collections import defaultdict

# Define the markdown file
markdown_file = 'markdown_files/all_tests.md'

# Create a directory for the markdown file
os.makedirs(os.path.dirname(markdown_file), exist_ok=True)

# Create a dictionary to group the subtests by their categories
tests_by_category = defaultdict(list)

# Iterate over all dictionaries in test_detector_dicts
for test_detector_dict in test_detector_dicts:
    # Iterate over the items in each dictionary
    for test, detectors in test_detector_dict.items():
        # Extract the relevant part of the test name
        test_category = str(test).split('.')[2]
        test_name = str(test).split(' ')[0].split('.')[-1]

        # Group the subtests by their categories
        tests_by_category[test_category].append((test_name, detectors))

# Open the markdown file
with open(markdown_file, 'w') as f:
    # Iterate over the categories
    for category, tests in tests_by_category.items():
        # Write the category to the file
        f.write(f'## {category}\n')

        # Write the subtests to the file
        for test_name, detectors in tests:
            f.write(f'- {test_name}\n')

            # Write the detectors to the file
            for detector in detectors:
                # Extract the relevant part of the detector name
                detector_name = '.'.join(str(detector).split(' ')[0].split('.')[1:])
                f.write(f'  - {detector_name}\n')

        # Add a newline for readability
        f.write('\n')

In [ ]:
# Update each dictionary with the result of calling get_detectors on it
for i in range(len(test_detector_dicts)):
    test_detector_dicts[i] = get_detectors(test_detector_dicts[i])

In [ ]:
test_detector_dicts

In [ ]:
import os

# Define the markdown file
markdown_file = 'markdown_files/all_tests.md'

# Create a directory for the markdown file
os.makedirs(os.path.dirname(markdown_file), exist_ok=True)

# Open the markdown file
with open(markdown_file, 'w') as f:
    # Iterate over the dictionary in test_detector_dicts
    for test, detectors in test_detector_dicts[0].items():
        # Extract the test name from the object string
        test_name = str(test).split(' ')[0].split('.')[-1]

        # Write the test name to the file
        f.write(f'## {test_name}\n')

        # Write the detectors to the file
        for detector in detectors:
            f.write(f'- {detector}\n')

        # Add a newline for readability
        f.write('\n')

In [ ]:

counts = [get_counts(dict) for dict in test_detector_dicts]

In [ ]:
counts

In [ ]:
harness_detector_counts= list(zip(harnesses, counts))

In [ ]:
harness_detector_counts

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Iterate over the zipped list
for harness, counter in harness_detector_counts:
    # Get the names of the detectors and their counts
    detectors = list(counter.keys())
    counts = list(counter.values())
    
    # Sort the detectors and counts by the counts
    detectors, counts = zip(*sorted(zip(detectors, counts), key=lambda x: x[1]))
    
    # Create a bar chart
    plt.figure(figsize=(10, 5))  # Set the figure size
    plt.barh(detectors, counts)  # Create a horizontal bar chart
    plt.title(str(harness))  # Set the title to the name of the harness
    plt.xlabel('Counts')  # Set the x-axis label
    plt.ylabel('Detectors')  # Set the y-axis label
    plt.xlim(0, max(counts) + 1)  # Set the x-axis limits
    plt.xticks(np.arange(0, max(counts) + 1, step=1))  # Set the x-axis ticks to integers
    plt.show()  # Display the plot

In [ ]:
from autoredteam.harnesses.dimension import * 
harnesses = [SecurityHarness, ToxicityHarness, PrivacyHarness, HallucinationHarness, RobustnessHarness, ToxicityHarness, StereotypeHarness, FairnessHarness, EthicsHarness]
harness_instances = [harness(agent) for harness in harnesses]

In [ ]:
from autoredteam.detectors.base import *
detectors = [Detector, HFDetector, StringAbsenceDetector, StringDetector, TriggerListAbsenceDetector, TriggerListDetector]
first_instance = detectors[0]()

# Get all the attributes and methods of the instance
attributes = dir(first_instance)

# Print the attributes
for attribute in attributes:
    print(attribute)

In [ ]:
from autoredteam.detectors.base import *
detectors = [Detector, StringDetector, TriggerListAbsenceDetector, TriggerListDetector]

# List of attributes to extract
attributes_to_extract = ['bcp47', 'description', 'detect', 'detectorname', 'name', 'precision', 'recall', 'tags', 'uri']

for DetectorClass in detectors:
    # Create an instance of the class
    instance = DetectorClass()

    # Extract the specified attributes
    for attribute in attributes_to_extract:
        if hasattr(instance, attribute):
            print(f'{attribute}: {getattr(instance, attribute)}')

In [ ]:
import inspect

# List of attributes to look for
attributes_to_look_for = ['name', 'description', 'detectors', 'goal', 'probe', 'tags', 'uri']

instances_data = []
for instance in test.test_instances:
    instance_dict = {}
    for attribute in attributes_to_look_for:
        if hasattr(instance, attribute):
            if attribute == 'detectors':
                # Get the class name of each detector and join them into a single string
                instance_dict[attribute] = ', '.join([type(detector).__module__ + '.' + type(detector).__name__ for detector in getattr(instance, attribute)])
            elif attribute == 'probe':
                # Get the class name of the probe
                probe = getattr(instance, attribute)
                instance_dict[attribute] = type(probe).__module__ + '.' + type(probe).__name__
            elif attribute == 'tags':
                # Join the tags into a single string
                instance_dict[attribute] = ', '.join(getattr(instance, attribute))
            else:
                instance_dict[attribute] = getattr(instance, attribute)
    instances_data.append(instance_dict)

print(instances_data)

In [ ]:
from tabulate import tabulate

# Convert the list of dictionaries to a markdown table
markdown_table = tabulate(instances_data, headers="keys", tablefmt="pipe")

# Add a title to the markdown
markdown_table = '# Security\n' + markdown_table

print(markdown_table)

In [ ]:
from autoredteam.harnesses.dimension import * 
from tabulate import tabulate

harnesses = [SecurityHarness, ToxicityHarness, PrivacyHarness, HallucinationHarness, RobustnessHarness, ToxicityHarness, StereotypeHarness, FairnessHarness, EthicsHarness]
harness_instances = [harness(agent) for harness in harnesses]

# List of attributes to look for
attributes_to_look_for = ['name', 'description', 'detectors', 'goal', 'probe', 'tags', 'uri']

for harness_instance in harness_instances:
    instances_data = []
    for instance in harness_instance.test_instances:
        instance_dict = {}
        for attribute in attributes_to_look_for:
            if hasattr(instance, attribute):
                if attribute == 'detectors':
                    # Get the class name of each detector and join them into a single string
                    instance_dict[attribute] = ', '.join([type(detector).__module__ + '.' + type(detector).__name__ for detector in getattr(instance, attribute)])
                elif attribute == 'probe':
                    # Get the class name of the probe
                    probe = getattr(instance, attribute)
                    instance_dict[attribute] = type(probe).__module__ + '.' + type(probe).__name__
                elif attribute == 'tags':
                    # Join the tags into a single string
                    instance_dict[attribute] = ', '.join(getattr(instance, attribute))
                elif attribute == 'description':
                    # Check if the description is a tuple, and if so, take the first element
                    description = getattr(instance, attribute)
                    if isinstance(description, tuple):
                        description = description[0]
                    instance_dict[attribute] = description
                else:
                    instance_dict[attribute] = getattr(instance, attribute)
        instances_data.append(instance_dict)

    # Convert the list of dictionaries to a markdown table
    markdown_table = tabulate(instances_data, headers="keys", tablefmt="pipe")

    # Add a title to the markdown
    markdown_table = '# ' + type(harness_instance).__name__ + '\n' + markdown_table

    print(markdown_table)

In [ ]:
from autoredteam.harnesses.dimension import * 
from tabulate import tabulate

def generate_documentation(attributes_to_look_for, save_path):
    harnesses = [SecurityHarness, ToxicityHarness, PrivacyHarness, HallucinationHarness, RobustnessHarness, ToxicityHarness, StereotypeHarness, FairnessHarness, EthicsHarness]
    harness_instances = [harness(agent) for harness in harnesses]

    with open(save_path, 'w') as f:
        for harness_instance in harness_instances:
            instances_data = []
            for instance in harness_instance.test_instances:
                instance_dict = {}
                for attribute in attributes_to_look_for:
                    if hasattr(instance, attribute):
                        if attribute == 'detectors':
                            # Get the class name of each detector and join them into a single string
                            instance_dict[attribute] = ', '.join([type(detector).__module__ + '.' + type(detector).__name__ for detector in getattr(instance, attribute)])
                        elif attribute == 'probe':
                            # Get the class name of the probe
                            probe = getattr(instance, attribute)
                            instance_dict[attribute] = type(probe).__module__ + '.' + type(probe).__name__
                        elif attribute == 'tags':
                            # Join the tags into a single string
                            instance_dict[attribute] = ', '.join(getattr(instance, attribute))
                        else:
                            instance_dict[attribute] = getattr(instance, attribute)
                instances_data.append(instance_dict)

            # Convert the list of dictionaries to a markdown table
            markdown_table = tabulate(instances_data, headers="keys", tablefmt="pipe")

            # Add a title to the markdown
            markdown_table = '# ' + type(harness_instance).__name__ + '\n' + markdown_table

            # Write the markdown table to the file
            f.write(markdown_table + '\n\n')

# Call the function with the attributes to look for and the save path
generate_documentation(['name', 'description', 'detectors', 'goal', 'probe', 'tags', 'uri'], 'vijil_detectors.md')

In [ ]:
import argparse
from autoredteam.harnesses.dimension import * 
from tabulate import tabulate

def generate_documentation(attributes_to_look_for, save_path):
    harnesses = [SecurityHarness, ToxicityHarness, PrivacyHarness, HallucinationHarness, RobustnessHarness, ToxicityHarness, StereotypeHarness, FairnessHarness, EthicsHarness]
    harness_instances = [harness(agent) for harness in harnesses]

    with open(save_path, 'w') as f:
        for harness_instance in harness_instances:
            instances_data = []
            for instance in harness_instance.test_instances:
                instance_dict = {}
                for attribute in attributes_to_look_for:
                    if hasattr(instance, attribute):
                        if attribute == 'detectors':
                            # Get the class name of each detector and join them into a single string
                            instance_dict[attribute] = ', '.join([type(detector).__module__ + '.' + type(detector).__name__ for detector in getattr(instance, attribute)])
                        elif attribute == 'probe':
                            # Get the class name of the probe
                            probe = getattr(instance, attribute)
                            instance_dict[attribute] = type(probe).__module__ + '.' + type(probe).__name__
                        elif attribute == 'tags':
                            # Join the tags into a single string
                            instance_dict[attribute] = ', '.join(getattr(instance, attribute))
                        elif attribute == 'description':
                            # Check if the description is a tuple, and if so, take the first element
                            description = getattr(instance, attribute)
                            if isinstance(description, tuple):
                                description = description[0]
                            instance_dict[attribute] = description
                        else:
                            instance_dict[attribute] = getattr(instance, attribute)
                instances_data.append(instance_dict)

            # Convert the list of dictionaries to a markdown table
            markdown_table = tabulate(instances_data, headers="keys", tablefmt="pipe")

            # Add a title to the markdown
            markdown_table = '# ' + type(harness_instance).__name__ + '\n' + markdown_table

            # Write the markdown table to the file
            f.write(markdown_table + '\n\n')

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Generate documentation for test instances.')
    parser.add_argument('attributes', type=str, nargs='+', help='List of attributes to generate documentation for.')
    parser.add_argument('save_path', type=str, help='Path to save the markdown file.')
    args = parser.parse_args()

    # Call the function with the attributes to look for and the save path
    generate_documentation(args.attributes, args.save_path)

In [ ]:
from garak.detectors.dan import DAN, DANJailbreak, DevMode, DUDE, STAN, AntiDAN, MarkdownLink
from tabulate import tabulate

# List of attributes to extract
attributes_to_extract = ['bcp47', 'description', 'detect', 'detectorname', 'name', 'precision', 'recall', 'tags', 'uri']

instances = [DAN, DANJailbreak, DevMode, DUDE, STAN, AntiDAN, MarkdownLink]

data = []

# Extract the specified attributes
for instance in instances:
    row = {'name': instance.__name__}
    for attribute in attributes_to_extract:
        if hasattr(instance, attribute):
            row[attribute] = getattr(instance, attribute)
    data.append(row)

# Convert the list of dictionaries to a markdown table
markdown_table = tabulate(data, headers="keys", tablefmt="pipe")

# Save the markdown table to a file
with open('output.md', 'w') as f:
    f.write(markdown_table)

In [ ]:
import pkgutil
import garak.detectors as detectors
from tabulate import tabulate

# List of attributes to extract
attributes_to_extract = ['detectorname', 'name', 'bcp47', 'description', 'detect', 'tags', 'uri']

# Get all modules in the garak.detectors package
detector_modules = list(pkgutil.iter_modules(detectors.__path__))

instances = []

# Import each module and add its classes to the instances list
for _, module_name, _ in detector_modules:
    module = __import__(f'garak.detectors.{module_name}', fromlist=[module_name])
    for attribute_name in dir(module):
        attribute = getattr(module, attribute_name)
        if isinstance(attribute, type) and attribute_name != 'Attempt':
            instances.append(attribute)

data = []

# Extract the specified attributes
for instance in instances:
    row = {'name': instance.__name__}
    for attribute in attributes_to_extract:
        if hasattr(instance, attribute):
            row[attribute] = getattr(instance, attribute)
    data.append(row)

# Convert the list of dictionaries to a markdown table
markdown_table = tabulate(data, headers="keys", tablefmt="pipe")

# Save the markdown table to a file
with open('new_output.md', 'w') as f:
    f.write(markdown_table)

In [ ]:
from garak.detectors.riskywords import SurgeProfanityAnimalReferences
this = SurgeProfanityAnimalReferences()
this.substrings

In [ ]:
import pkgutil
import garak.detectors as detectors
from tabulate import tabulate
import types
import json

class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, type):
            return str(obj)
        return super().default(obj)

# List of attributes to extract
attributes_to_extract = ['detectorname', 'name', 'bcp47', 'description', 'detect', 'tags', 'uri', 'substrings']

# Get all modules in the garak.detectors package
detector_modules = list(pkgutil.iter_modules(detectors.__path__))

instances = []

# Import each module and add its classes to the instances list
for _, module_name, _ in detector_modules:
    module = __import__(f'garak.detectors.{module_name}', fromlist=[module_name])
    for attribute_name in dir(module):
        attribute = getattr(module, attribute_name)
        if isinstance(attribute, type) and attribute_name != 'Attempt':
            try:
                instances.append(attribute())
            except Exception:
                pass

data = []

# Extract the specified attributes
for instance in instances:
    row = {'name': instance.__class__.__name__}
    if row['name'] not in ['Detector', 'StringDetector', 'TriggerListDetector', 'defaultdict']:
        for attribute in attributes_to_extract:
            if hasattr(instance, attribute):
                attr_value = getattr(instance, attribute)
                if isinstance(attr_value, types.MethodType):
                    row[attribute] = f'{instance.__class__.__name__}.{attribute}'
                elif isinstance(attr_value, list):
                    row[attribute] = ', '.join(attr_value)
                else:
                    row[attribute] = attr_value
            else:
                if attribute == 'substrings':
                    row[attribute] = 'N/A'
        data.append(row)

# Convert the list of dictionaries to a markdown table
markdown_table = tabulate(data, headers="keys", tablefmt="pipe")

# Save the markdown table to a file
with open('detectors.md', 'w') as f:
    f.write(markdown_table)

In [ ]:
class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        return str(obj)

# Save the data to a JSON file
with open('detector.json', 'w') as f:
    json.dump(data, f, cls=CustomEncoder)


In [ ]:
this.substrings

In [ ]:
from autoredteam.detectors import adultdata, advstereo, base, hallucination, paraphrase

In [ ]:
def write_detector_info(modules_to_inspect):
    # Initialize a list to store the detector info
    detector_info = []

    # Initialize a set to store the names of the saved detectors
    saved_detectors = set()

    # Iterate over each module
    for module_name in modules_to_inspect:
        # Import the module
        module = importlib.import_module(module_name)

        # Get all classes in the module
        classes = inspect.getmembers(module, inspect.isclass)

        # Iterate over each class
        for class_name, class_ in classes:
            # Get all subclasses of the class
            subclasses = class_.__subclasses__()

            # Iterate over each subclass
            for subclass in subclasses:
                # Check if the detector has already been saved
                if subclass.__name__ in saved_detectors:
                    continue

                # Add the detector to the set of saved detectors
                saved_detectors.add(subclass.__name__)

                # Try to create an instance of the subclass
                try:
                    instance = subclass()
                except TypeError:
                    instance = None

                # Get the subclass info
                info = {
                    "Detector": subclass.__name__,
                    "Module": subclass.__module__,
                    "Description": inspect.getdoc(subclass),
                    "Substrings": instance.substrings if instance and hasattr(instance, 'substrings') else "N/A"
                }
                # Add the subclass info to the detector info list
                detector_info.append(info)

    # Write the detector info to a JSON file
    with open('art_detectors.json', 'w') as f:
        json.dump(detector_info, f, indent=4)

    # Write the detector info to a Markdown file
    with open('art_detectors.md', 'w') as f:
        # Write the table headers
        f.write("| Detector | Module | Description | Substrings |\n")
        f.write("| -------- | ------ | ----------- | ---------- |\n")

        # Write the table data
        for info in detector_info:
            f.write(f"| {info['Detector']} | {info['Module']} | {info['Description']} | {info['Substrings']} |\n")

# List of modules to inspect
modules_to_inspect = ['autoredteam.detectors.adultdata', 'autoredteam.detectors.advstereo', 
                      'autoredteam.detectors.hallucination', 
                      'autoredteam.detectors.paraphrase']

write_detector_info(modules_to_inspect)

In [ ]:
import inspect
import importlib
import json

def get_module_info(module_name):
    # Import the module
    module = importlib.import_module(module_name)

    # Get all classes in the module
    classes = inspect.getmembers(module, inspect.isclass)

    # Initialize a list to store the class info
    class_info = []

    # Iterate over each class
    for class_name, class_ in classes:
        # Get the class info
        info = {
            "Detector": class_name,
            "Module": class_.__module__,
            "Description": inspect.getdoc(class_),
            "Substrings": "N/A"
        }
        # Add the class info to the class info list
        class_info.append(info)

    return class_info

# List of modules to inspect
modules_to_inspect = ['garak.detectors.base']

# Iterate over each module
for module_name in modules_to_inspect:
    # Get the module info
    module_info = get_module_info(module_name)

    # Write the module info to a JSON file
    with open('module_info.json', 'w') as f:
        json.dump(module_info, f, indent=4)

    # Write the module info to a Markdown file
    with open('module_info.md', 'w') as f:
        # Write the table headers
        f.write("| Detector | Module | Description | Substrings |\n")
        f.write("| -------- | ------ | ----------- | ---------- |\n")

        # Write the table data
        for info in module_info:
            f.write(f"| {info['Detector']} | {info['Module']} | {info['Description']} | {info['Substrings']} |\n")

In [ ]:
!pip install scikit-learn

In [ ]:
import pandas as pd
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from io import StringIO

# Path to your markdown file
md_file_path = 'test_detector_pairing.md'

# Initialize an empty DataFrame
data = pd.DataFrame(columns=['harness', 'name', 'detectors'])

# Read markdown file and parse data
with open(md_file_path, 'r') as f:
    lines = f.readlines()
    harness = None
    for i, line in enumerate(lines):
        if line.startswith('#'):
            harness = line.strip().lstrip('# ')
        elif line.startswith('| name'):
            end = [j for j in range(i, len(lines)) if lines[j].startswith('|:---')]
            md_data = ''.join(lines[i:end[0]+3])  # Adjusted this line to read all rows of the table
            df = pd.read_csv(StringIO(md_data), sep="|")
            df.columns = df.columns.str.strip()
            df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
            df = df.assign(detectors=df['detectors'].str.split(',')).explode('detectors')
            df['detectors'] = df['detectors'].str.strip()
            df['harness'] = harness
            data = pd.concat([data, df])

# Create a linkage matrix
le = LabelEncoder()
data['label'] = data['harness'] + ' -> ' + data['name'] + ' -> ' + data['detectors']
linkage_matrix = linkage(le.fit_transform(data['label']).reshape(-1, 1), 'single')

# Create the dendrogram
plt.figure(figsize=(10, 7))
dendrogram(linkage_matrix, labels=data['label'].values, orientation='right')
plt.show()